In [ ]:
import pandas as pd
import numpy as np
import concurrent.futures
import torch
from PIL import Image

In [ ]:
data1 = pd.read_csv('/media/hdd2/navaneeth/Deep-Learning-Projects/pytorch/color/color_bags.csv')
data2 = pd.read_csv('/media/hdd2/navaneeth/Deep-Learning-Projects/pytorch/color/color_tops.csv')
data3 = pd.read_csv('/media/hdd2/navaneeth/Deep-Learning-Projects/pytorch/color/color_pants.csv')

In [ ]:
"""import requests

def download_image(url, file_name):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        print('Downloading ' + file_name)
        r.raw.decode_content = True
        with open(file_name, 'wb') as f:
            f.write(r.content)
        return file_name
    else:
        return None

with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, row in data1.iterrows():
        file_name = 'bags_images/' + str(i) + '.jpg'
        url = row['image_url']
        executor.submit(download_image, url, file_name)
    for i, row in data2.iterrows():
        file_name = 'tops_images/' + str(i) + '.jpg'
        url = row['image_url']
        executor.submit(download_image, url, file_name)
    for i, row in data3.iterrows():
        file_name = 'pants_images/' + str(i) + '.jpg'
        url = row['image_url']
        executor.submit(download_image, url, file_name) """

In [ ]:
data1=data1[data1['Category']=='Bags']
data2=data2[data2['Category']=='Tops']
data3=data3[data3['Category']=='Pants']


In [ ]:
data1.drop(['image_url'], axis=1, inplace=True)
data2.drop(['image_url'], axis=1, inplace=True)
data3.drop(['image_url'], axis=1, inplace=True)
data1.drop(['Category'], axis=1, inplace=True)
data2.drop(['Category'], axis=1, inplace=True)
data3.drop(['Category'], axis=1, inplace=True)

In [ ]:
def preprocess_image(file):
    img = Image.open(file)
    return img

with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, row in data1.iterrows():
        file_name = 'bags_images/' + str(i) + '.jpg'
        data1.at[i, 'image_data'] = executor.submit(
            preprocess_image, file_name)
    for i, row in data2.iterrows():
        file_name = 'tops_images/' + str(i) + '.jpg'
        data2.at[i, 'image_data'] = executor.submit(
            preprocess_image, file_name)
    for i, row in data3.iterrows():
        file_name = 'pants_images/' + str(i) + '.jpg'
        data3.at[i, 'image_data'] = executor.submit(
            preprocess_image, file_name)

data1['image_data'] = data1['image_data'].apply(lambda x: x.result())
data2['image_data'] = data2['image_data'].apply(lambda x: x.result())
data3['image_data'] = data3['image_data'].apply(lambda x: x.result())

In [ ]:
print(data1.head())
print(data2.head())
print(data3.head())

In [ ]:
# plot 10 random images from each category along with their Base Color as title
import matplotlib.pyplot as plt
import random

random.seed(31)
fig, ax = plt.subplots(3, 10, figsize=(20, 10))
for i in range(10):
    x = random.randint(0, len(data1))
    y = random.randint(0, len(data2))
    z = random.randint(0, len(data3))
    ax[0, i].imshow(data1.iloc[x]['image_data'])
    ax[0, i].set_title(data1.iloc[x]['Base Color'])
    ax[1, i].imshow(data2.iloc[y]['image_data'])
    ax[1, i].set_title(data2.iloc[y]['Base Color'])
    ax[2, i].imshow(data3.iloc[z]['image_data'])
    ax[2, i].set_title(data3.iloc[z]['Base Color'])
    ax[0, i].axis('off')
    ax[1, i].axis('off')
    ax[2, i].axis('off')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

train_data1, temp_data1 = train_test_split(data1, test_size=0.3, random_state=31)
valid_data1, test_data1 = train_test_split(
    temp_data1, test_size=0.5, random_state=31)

train_data2, temp_data2 = train_test_split(data2, test_size=0.3, random_state=31)
valid_data2, test_data2 = train_test_split(
    temp_data2, test_size=0.5, random_state=31)

train_data3, temp_data3 = train_test_split(data3, test_size=0.3, random_state=31)
valid_data3, test_data3 = train_test_split(
    temp_data3, test_size=0.5, random_state=31)

print("Bags: ")
print("Training set size:", len(train_data1))
print("Validation set size:", len(valid_data1))
print("Test set size:", len(test_data1))

print("Tops: ")
print("Training set size:", len(train_data2))
print("Validation set size:", len(valid_data2))
print("Test set size:", len(test_data2))

print("Pants: ")
print("Training set size:", len(train_data3))
print("Validation set size:", len(valid_data3))
print("Test set size:", len(test_data3))